- 테스트 기사 url = https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=056&aid=0010661268

In [1]:
# 필요한 모듈과 라이브러리 로딩

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import math
import numpy
import pandas as pd
import os
import random

In [ ]:
# 사용자에게 검색어 키워드 입력받고 저장할 폴더와 파일명을 설정
print('=' * 80)
print('뉴스 기사의 댓글 정보 수집하기')
print('=' * 80)
print('\n')

txt = '뉴스기사댓글'
url = input('댓글을 크롤링할 뉴스의 url을 입력하세요: ')
cnt = int(input('크롤링 할 건수는 몇 건입니까(10단위로 입력 요망): ')) 
page_cnt = math.ceil(cnt/20)

f_dir = input('파일을 저장할 폴더명만 쓰세요: ')

# 저장될 파일위치와 이름을 지정합니다 
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_yeay, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

os.makedirs(f_dir+s+'-'+txt)
os.chdir(f_dir+s+'-'+txt)

ff_name = f_dir+s+'-'+txt+'\\'+s+txt+'.txt'
fc_name = f_dir+s+'-'+txt+'\\'+s+txt+'.csv'
fx_name = f_dir+s+'-'+txt+'\\'+s+txt+'.xls'

# 크롬 드라이버를 사용하여 웹 브라우저 실행
s_time = time.time()

path = './chromedriver.exe'
driver = webdriver.Chrome(path)
driver.get(url)
time.sleep(5)

# 현재 총 리뷰 건수를 확인하여 사용자의 요청건수와 비교 후 동기화
html = driver.page_source
soup = BeautifulSoup(html,'html.parser')

result = soup.find('div',class_ = 'u_cbox_head').find('span','u_cbox_count')
result2 = result.get_text()

print('=' * 80)
result3 = result2.replace(',','')
result4 = re.search('\d', result3)
search_cnt = int(result4.group())

if cnt > search_cnt:
    cnt =search_cnt
    
print('전체 검색 결과 건수:', search_cnt, '건')
print('실제 최종 출력 건수:', cnt)
print('실제 출력될 페이지 수:', page_cnt)

# 사용자가 요청한 건수가 많을 경우 리뷰 더보기 버튼을 클릭한다.
# 댓글 중 일부 내용만 보일 경우 전체 내용을 수집하기
# 최초 10건 수집후 댓글 더보기 버튼 클릭
# 아래 버튼을 눌러 첫 화면에 총 20건의 댓글이 나오게 만듬
driver.find_element_by_xpath('''//*[@id="cbox_module"]/div/div[9]/a/span[1]''').click()
time.sleep(3)

# 삭제된 댓글이나 항목이 있을 경우 예외처리하기
# 20건 출력되어 있는 현재 페이지 리뷰와 점수 등 내용 수집
writer_id2 = []
review2 = []
writer_date2 = []
gogam = []
gogam_0 = []
gogam_1 = []

if cnt <=20:
    f = open(ff_name, 'a', encoding='utf-8')
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    count = 0
    
    reple_result = soup.find('div', class_ = 'u_cbox_text_wrap').find('ul')
    slist = reple_result.find_all('li')
    
    for li in slist:
        count += 1
        print('\n')
        print('%s 번째 댓글 수집 중=================' %count)
        
        writer_id = li.find('span',class_='u_cbox_nick').get_text()
        print('1. 작성자 ID:', writer_id)
        f.write('\n')
        f.write('총 %s 건 중 %s 번째 리뷰 데이터를 수집합니다.=================' %(cnt, count) + '\n')
        f.write('1. 작성자 ID:' + writer_id + '\n')
        writer_id2.append(writer_id)
        
        try:
            review = li.find('span', class_='u_cbox_contents').get_text()
        except AttributeError:
            review = '작성자에 의해 삭제된 댓글입니다.'
            print('2. 리뷰: ', review)
        else:
            print('2.리뷰: ', review )
        f.write('2. 리뷰: ' + review + '\n')
        review2.append(review)
        
        writer_date = li.find('span', class_='u_cbox_date').get_text()
        print('3. 작성일자: ', write_date)
        f.write('3. 작성일자: ' + writer_date + '\n')
        writer_date2.append(writer_date)
        
        gogam = 